In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import pickle
import matplotlib.pyplot as plt
import sqlite3
import warnings
import random
import sqlalchemy
from math import comb

# Make sure these are in the filepath.
import factorization
import sampling

## Functions.
These functions implement and evaluate the link prediction process.

In [3]:
# Remove perc edges, for evaluation.
# Does not mutate G.

# DOES NOT WORK FOR DIRECTED GRAPHS.
def remove_edges(G, perc=0.2):
    
    size = G.number_of_edges()
    
    # Num edges to remove.
    N = np.floor(0.2*size)
    
    # Initialize output.
    G_removed = G.copy()
    removed_edges = []
    bridges = []
    j = 0
    
    while (len(removed_edges) < N) and (j < size):
        
        # Don't choose edges from the list of known bridges.
        candidate_edges = list(set(G_removed.edges()) - set(bridges))
        edge = random.choice(candidate_edges)
        
        # Check if the selected edge is a bridge.
        G_removed.remove_edge(edge[0], edge[1])
        is_bridge = not nx.has_path(G_removed, edge[0], edge[1])
        
        if is_bridge:
            # If edge is a bridge, add it back to the graph and pass.
            # Also add it to list of bridges so it isn't selected again.
            G_removed.add_edge(edge[0], edge[1])
            bridges.append(edge)
        else:
            # Otherwise, don't add the edge back and append it to output.
            removed_edges.append(edge)
        
        j = j + 1
    
    # Return a new graph with edges remove, as well as what was removed.
    return G_removed, removed_edges

In [4]:
# This function calculates embeddings for G.
def calc_embeddings(G, directed=False, alg="factorization", sim="autocovariance"):
    # G -- nx.Graph, or nx.DiGraph (if directed==True)
    # Directed -- t/f
    # alg -- embedding algorithm -- "factorization", "sampling"
    # sim -- similarity metric -- "autocovariance", "PMI"
    warnings.filterwarnings('ignore')
    
    if alg == "factorization":
        
        # factorization.embed(G, dimensions=128, markov_time, directed, similarity, average_similarity)
        emb = factorization.embed(G, 128, 3, directed, sim, False)
        
    elif alg == "sampling":
        
        # sampling.embed(G, dimensions=128, markov_time, None, directed, similarity, average_similarity,
        #       lr, iter, early_stop, batch_size, neg, walks, walk_length, damp, workers)
        emb = sampling.embed(G, 128, 3, None, directed, sim, False,
                            6e-3, 50, 5, 100, 1, 10, 80, 0.85, 32)
        
    return emb[:, ~np.isnan(emb).any(axis=0)]

In [12]:
# Given a list of embeddings, return a list of top k predicted edges.
def link_prediction(G, emb, k):
    
    existing_edges = [(min(edge), max(edge)) for edge in G.edges()]
    vertices = sorted(list(G.nodes()))
    
    candidate_edges = [(u, v) for u in vertices for v in vertices]
    candidate_edges = [edge for edge in candidate_edges if
                       (edge[0] < edge[1])]
    candidate_edges = [edge for edge in candidate_edges if
                       (edge not in existing_edges)]
    
    dic = {}
    for j in range(len(vertices)):
        dic[vertices[j]] = emb[j, :]
        
    # I'm not sure if this should be sorted by asc or desc.
    predictions = sorted(candidate_edges,
                         key=lambda x: np.dot(dic[x[0]], dic[x[1]]), reverse=True)
    
    return predictions[0:k]

In [6]:
# Do everything: Remove edges, link prediction, and evaluate.
def eval_model(G, k, get_all_preds=False):
    G_removed, removed_edges = remove_edges(G)
    order = len(G_removed.nodes())
    size = len(G_removed.edges())
    
    # If this is true, return the whole list of predictions rather than the confusion matrix.
    if get_all_preds:
        k = comb(order, 2) - size
    
    # Make predictions.
    emb = calc_embeddings(G_removed)
    predictions = link_prediction(G_removed, emb, k)
    
    if get_all_preds:
        return predictions
            
    # Evaluate against removed_edges.
    removed_edges = [(min(edge), max(edge)) for edge in removed_edges]
    TP = len([edge for edge in predictions if (edge in removed_edges)])
    FP = len(predictions) - TP
    FN = len(removed_edges) - TP
    TN = comb(order, 2) - size - TP - FP - FN
            
    return (TP, FP, FN, TN)

## Experiments.

In [14]:
# Experiments for shortened graph.

# Total number of prediction is k * #Clusters. In this case, 5*5 = 25.
k = 5
TP = 0
TF = 0

for j in range(5):
    path = "metis_graphs/G_short_partitions/G_short_partition" + str(j) +".pk"
    with open(path, "rb") as file:
        subgraph = pickle.load(file)
    CMatrix = eval_model(subgraph, k=k)
    TP = TP + CMatrix[0]
    TF = TF + CMatrix[1]

prec = TP/(TP+TF)
print("Precision for METIS clustered short graph: " + str(prec))

Precision for METIS clustered short graph: 0.44


In [15]:
# Experiments for full graph.
# This takes many hours!!

# Total number of prediction is k * #Clusters. In this case, 5*10 = 50.
k = 5
TP = 0
TF = 0

for j in range(10):
    path = "metis_graphs/G_full_partitions/G_full_partition" + str(j) +".pk"
    with open(path, "rb") as file:
        subgraph = pickle.load(file)
    CMatrix = eval_model(subgraph, k=k)
    TP = TP + CMatrix[0]
    TF = TF + CMatrix[1]

prec = TP/(TP+TF)
print("Precision for METIS clustered full graph: " + str(prec))

Precision for METIS clustered full graph: 0.64


In [ ]:
# FINAL PREDICTIONS FOR FULL GRAPH.
# This cell also takes a while!